In [3]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tqdm import tqdm_notebook

In [4]:
init_notebook_mode(connected=True)

In [40]:
df_all = pd.read_csv('00_data/elections_round2_all_v2.csv')
print(df_all.shape)
df_all.head()

(29608, 68)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,ЗеленськийВолодимир_inflow_ТимошенкоЮлія,ЗеленськийВолодимир_inflow_other_cands,ПорошенкоПетро_inflow_others,ПорошенкоПетро_inflow_БойкоЮрій,ПорошенкоПетро_inflow_ВілкулОлександр,ПорошенкоПетро_inflow_ГриценкоАнатолій,ПорошенкоПетро_inflow_ЛяшкоОлег,ПорошенкоПетро_inflow_СмешкоІгор,ПорошенкоПетро_inflow_ТимошенкоЮлія,ПорошенкоПетро_inflow_other_cands
0,50130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103.0,602,1004,90,...,117.521042,34.074148,129.0,8.272545,2.843687,18.871743,11.633267,31.022044,43.689379,12.667335
1,50131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15.0,822,1295,11,...,140.054795,48.657534,162.0,11.589041,8.136986,23.917808,11.342466,36.246575,52.520548,18.246575
2,50132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4.0,484,808,4,...,74.846361,27.347709,80.0,8.840970,2.156334,10.350404,7.547170,20.485175,22.425876,8.194070
3,50133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14.0,933,1504,11,...,140.350474,50.428958,185.0,13.768606,7.259811,23.531800,15.270636,46.562923,57.828146,20.778078
4,50134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0.0,108,132,0,...,16.400000,5.046154,16.0,0.246154,0.246154,2.953846,0.738462,3.446154,6.400000,1.969231


In [41]:
#credentials needed
py.sign_in('okuchma', '1X4AOENqX8gafp4qNWcS')

In [42]:
candidates = ['ЗеленськийВолодимир','ПорошенкоПетро']

base_cols = ['district_number', 'district_gen_numeric']

main_cands = ['БойкоЮрій',
             'ВілкулОлександр',
             'ГриценкоАнатолій',
             'ЛяшкоОлег',
             'СмешкоІгор',
             'ТимошенкоЮлія',
             'other_cands']

In [43]:
no_vote = 'не голосували'
indices = [i for i in range(len(candidates+main_cands+[no_vote]+candidates+[no_vote]))]
indices = [i for i in zip([outflow for outflow in (candidates+main_cands+[no_vote])]+
                          [inflow for inflow in (candidates+[no_vote])],indices)]

indices

[('ЗеленськийВолодимир', 0),
 ('ПорошенкоПетро', 1),
 ('БойкоЮрій', 2),
 ('ВілкулОлександр', 3),
 ('ГриценкоАнатолій', 4),
 ('ЛяшкоОлег', 5),
 ('СмешкоІгор', 6),
 ('ТимошенкоЮлія', 7),
 ('other_cands', 8),
 ('не голосували', 9),
 ('ЗеленськийВолодимир', 10),
 ('ПорошенкоПетро', 11),
 ('не голосували', 12)]

In [56]:
proper_names = {'ЗеленськийВолодимир':'Володимир Зеленський',
                'ПорошенкоПетро':'Петро Порошенко',
                'БойкоЮрій':'Юрій Бойко',
                'ВілкулОлександр':'Олександр Вілкул',
                'ГриценкоАнатолій':'Анатолій Гриценко',
                'ЛяшкоОлег':'Олег Ляшко',
                'СмешкоІгор':'Ігор Смешко',
                'ТимошенкоЮлія':'Юлія Тимошенко',
                'other_cands':'інші кандидати',
                'не голосували':'не голосували'}

labels = [proper_names[i[0]] for i in indices]
print(labels)

outflows = []
for index in indices[:10]:
    outflows += [index for i in range(3)]
    
inflows = [] 
for index in range(len(indices[:10])):
    inflows += indices[-3:]
    
colors_dict = {'Володимир Зеленський':'#3ec258',
               'Петро Порошенко':'#c2443e',
               'не голосували':'#666666'}

colors = [colors_dict[label] if label in colors_dict else '#ffdb83' for label in labels]

print(len(outflows),len(inflows))
source = [outflow[1] for outflow in outflows]
target = [inflow[1] for inflow in inflows]

['Володимир Зеленський', 'Петро Порошенко', 'Юрій Бойко', 'Олександр Вілкул', 'Анатолій Гриценко', 'Олег Ляшко', 'Ігор Смешко', 'Юлія Тимошенко', 'інші кандидати', 'не голосували', 'Володимир Зеленський', 'Петро Порошенко', 'не голосували']
30 30


In [57]:
for candidate in candidates:
    df_all[candidate+'_inflow_'+candidate] = df_all[candidate] - df_all[candidate+'_stayed_home'] - df_all[candidate+'_outflow']

df_all[candidates[0]+'_inflow_'+candidates[1]] = df_all[candidates[0]+'_inflow']
df_all[candidates[1]+'_inflow_'+candidates[0]] = df_all[candidates[1]+'_inflow']

df_all.columns

Index(['district_number', 'district_gen_numeric', 'district_general',
       'district_location', '1.К-сть одержаних бюлетенів',
       '2.К-сть виборців, внесених до списку_x',
       '3.К-сть виборців, внесених до витягу для голос. за місцем перебув.',
       '4.К-сть невикор. бюлетенів',
       '5.К-сть виборців, які отримали бюлетені у приміщ.',
       '6.К-сть виборців, які отримали бюлетені за місцем перебув.',
       '7.Загальна к-сть виборців, які отримали бюлетені',
       '8.К-сть бюлетенів, що не підлягають врахуванню',
       '9.К-сть виборців, які взяли участь у голосуванні',
       '10.К-сть бюлетенів, визнаних недійсними', 'ЗеленськийВолодимир_round2',
       'ПорошенкоПетро_round2', 'Дата та час протоколу',
       '2.К-сть виборців, внесених до списку', 'reg_voters_change',
       'reg_voters_change_abs', 'participation_round2', 'participation_round1',
       'ЗеленськийВолодимир', 'ПорошенкоПетро', 'БойкоЮрій', 'ВілкулОлександр',
       'ГриценкоАнатолій', 'ЛяшкоОлег',

In [58]:
def count_values(outflow,inflow):
    outflow_name = outflow[0].replace('_outflow','')
    inflow_name = inflow[0].replace('_inflow','')
    no_vote1_outflow = df_all['2.К-сть виборців, внесених до списку'].sum()-df_all['voted_round1'].sum()-df_all['mobilized'].sum()
    
    print(f'From candidate {outflow_name} to candidate {inflow_name}')
    print(inflow_name+'_inflow_'+outflow_name)
    
    if (outflow_name==no_vote)|(inflow_name==no_vote):
        if (outflow_name==no_vote)&(inflow_name==no_vote):
            print(no_vote1_outflow)
            values.append(no_vote1_outflow)
        elif outflow_name==no_vote:
            print(df_all[inflow_name+'_mobilized'].sum())
            values.append(df_all[inflow_name+'_mobilized'].sum())
        else:
            print(df_all[outflow_name+'_stayed_home'].sum())
            values.append(df_all[outflow_name+'_stayed_home'].sum())
    else:
        print(df_all[inflow_name+'_inflow_'+outflow_name].sum())
        values.append(df_all[inflow_name+'_inflow_'+outflow_name].sum())
    
values = []
for index,value in enumerate(outflows):
    count_values(outflows[index],inflows[index])
sum(values)

From candidate ЗеленськийВолодимир to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_ЗеленськийВолодимир
5642562.290594876
From candidate ЗеленськийВолодимир to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_ЗеленськийВолодимир
115.70940512477289
From candidate ЗеленськийВолодимир to candidate не голосували
не голосували_inflow_ЗеленськийВолодимир
1053.0
From candidate ПорошенкоПетро to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_ПорошенкоПетро
4746.95388360719
From candidate ПорошенкоПетро to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_ПорошенкоПетро
2933936.465560837
From candidate ПорошенкоПетро to candidate не голосували
не голосували_inflow_ПорошенкоПетро
42258.580555555556
From candidate БойкоЮрій to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_БойкоЮрій
1903463.036981687
From candidate БойкоЮрій to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_БойкоЮрій
160537.30516586313
From candidate БойкоЮрій to candidate не голосували
не голосували_inf

29655036.000000004

In [59]:
values_share = [value/sum(values) for value in values]
values_share

[0.19027332459130636,
 3.901846725958211e-06,
 3.550830287307693e-05,
 0.0001600724370594994,
 0.09893552196533624,
 0.0014250052016647544,
 0.06418683952977453,
 0.005413492169284944,
 0.003392646745431002,
 0.023109884553822894,
 0.001910016179562224,
 0.001099507105986849,
 0.02918589400690318,
 0.01062736409073151,
 0.003732020925189429,
 0.02654707833030712,
 0.00519644947988879,
 0.002754725362113476,
 0.02871185097991221,
 0.00628582547731595,
 0.0030572598988008472,
 0.06282196204942346,
 0.01479994052036019,
 0.006779067494218539,
 0.022043198764841822,
 0.00648940692652046,
 0.0026230316376112065,
 0.0035304210450026845,
 0.0011290573653421887,
 0.3737397250166885]

In [61]:
np_source = np.array(source)
np_values = np.array(values)
np_maxes = np.array([])

for triplet in range(np_source.max()+1):
    np_maxes = np.append(np_maxes,np_values[np.where(np_source==triplet)].max())

color_links = np.array(['rgba(204,204,204,0.3)' for i in range(len(np_source))])
color_links[np.where(np.isin(np_values,np_maxes))[0]]='rgba(204,204,204,1)'

data = {'type':'sankey',
       'node':{'label':labels,
               'color':colors,
               'pad':10,
               'line':{'width':0},
               'hovertemplate': "<extra>%{value:.1%}</extra> від усіх зареєстрованих виборців"
              },
       'link':{'source':source,
              'target':target,
              'value':values_share,
              'color':color_links,
              'hovertemplate': "<extra>%{value:.2%}</extra> від усіх зареєстрованих виборців"}
       }

fig = {'data':[data]}
py.iplot(fig, validate=False)
#iplot(fig, validate=False)

C:\Users\Ostap\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [39]:
['ЗеленськийВолодимир', 'ПорошенкоПетро',
'ЗеленськийВолодимир_stayed_home', 'ПорошенкоПетро_stayed_home',
'ЗеленськийВолодимир_outflow', 'ПорошенкоПетро_outflow',
'ЗеленськийВолодимир_inflow', 'ПорошенкоПетро_inflow',
'ЗеленськийВолодимир_inflow_others','ПорошенкоПетро_inflow_others',
'ЗеленськийВолодимир_mobilized', 'ПорошенкоПетро_mobilized']

candidates = ['Зеленський','Порошенко']
stayed_home = [df_all['ЗеленськийВолодимир_stayed_home'].sum()*(-1),df_all['ПорошенкоПетро_stayed_home'].sum()*(-1)]
outflow = [df_all['ЗеленськийВолодимир_outflow'].sum()*(-1),df_all['ПорошенкоПетро_outflow'].sum()*(-1)]
base = [df_all['ЗеленськийВолодимир'].sum(),df_all['ПорошенкоПетро'].sum()]
mobilization = [df_all['ЗеленськийВолодимир_mobilized'].sum(),df_all['ПорошенкоПетро_mobilized'].sum()]
inflow_competitor = [df_all['ЗеленськийВолодимир_inflow'].sum(),df_all['ПорошенкоПетро_inflow'].sum()]
inflow_others = [df_all['ЗеленськийВолодимир_inflow_others'].sum(),df_all['ПорошенкоПетро_inflow_others'].sum()]

labels_bar = ['0','2 млн','4 млн','6 млн','8 млн','10 млн']
tickvals = [0,2*10**6,4*10**6,6*10**6,8*10**6,10*10**6]

trace1 = go.Bar(
    y=candidates,
    x=stayed_home,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0)','rgba(194,68,62,0)']
    )
)

trace2 = go.Bar(
    y=candidates,
    x=outflow,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0)','rgba(194,68,62,0)']
    )
)

trace3 = go.Bar(
    y=candidates,
    x=base,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,1)','rgba(194,68,62,1)']
    )
)

trace4 = go.Bar(
    y=candidates,
    x=mobilization,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.8)','rgba(194,68,62,0.8)']
    )
)

trace5 = go.Bar(
    y=candidates,
    x=inflow_competitor,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.6)','rgba(194,68,62,0.6)']
    )
)

trace6 = go.Bar(
    y=candidates,
    x=inflow_others,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.6)','rgba(194,68,62,0.6)']
    )
)

data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = go.Layout(
    barmode='stack',
    showlegend=False,
    xaxis=go.layout.XAxis(
        ticktext=labels_bar,
        tickvals=tickvals
    )
)

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig)
iplot(fig)